In [39]:
import json
from transformers import PreTrainedTokenizerFast, CamembertForTokenClassification, AutoModelForTokenClassification,TrainingArguments,DataCollatorForTokenClassification,Trainer
import yaml
import re
from datasets import load_dataset,Dataset, load_metric
import evaluate
import datasets
from sklearn.model_selection import train_test_split
import importlib
import ast
import pandas as pd
import preprocess
import accelerate

In [3]:
importlib.reload(preprocess)

<module 'preprocess' from '/home/onyxia/work/NLP-project/preprocess.py'>

In [4]:
literal2tag=preprocess.literal2tag

In [5]:
df=pd.read_csv('data.csv')

In [6]:
dataset=load_dataset('csv',data_files='data.csv',num_proc=10)

In [7]:
features=datasets.Features(
                {
                    "text": datasets.Sequence(datasets.Value("string")),
                    "labels": datasets.Sequence(
                        datasets.features.ClassLabel(
                            num_classes=len(literal2tag),
                            names=literal2tag,
                        )
                    ),  
                    
                }
            )

In [8]:
features

{'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['age', 'birth_date', 'civil_status', 'education_level', 'employer', 'firstname', 'link', 'lob', 'maiden_name', 'nationality', 'observation', 'occupation', 'surname', 'surname_household'], id=None), length=-1, id=None)}

In [9]:
dataset = dataset['train'].map(lambda example: {"text": ast.literal_eval(example["text"]), "labels": ast.literal_eval(example["labels"])}, features=features)

In [10]:
dataset=dataset.train_test_split(test_size=0.2)

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 18443
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 4611
    })
})

In [12]:
tokenizer=PreTrainedTokenizerFast.from_pretrained('distilbert-base-uncased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [13]:
tokenizer_french=PreTrainedTokenizerFast.from_pretrained('camembert-base')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizerFast'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [14]:
tokenizer_french.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [15]:
label_all_tokens = False
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], padding=True, is_split_into_words=True,return_tensors='pt')
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
tokenized_datasets=dataset.map(tokenize_and_align_labels, batched=True,num_proc=10)

Map (num_proc=10):   0%|          | 0/18443 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/4611 [00:00<?, ? examples/s]

In [17]:
model=AutoModelForTokenClassification.from_pretrained('camembert-base', num_labels=14)

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18443
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4611
    })
})

In [19]:
#model(input_ids=tokenized_datasets['train']['input_ids'][0:1],attention_mask=tokenized_datasets['train']['attention_mask'][0:1],token_type_ids=tokenized_datasets['train']['token_type_ids'][0:1])

In [46]:
args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    push_to_hub=False,
    output_dir='/home/onyxia/work'
)

In [21]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [22]:
data_collator

DataCollatorForTokenClassification(tokenizer=PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

In [23]:
metric1 = evaluate.load("precision")
metric2 = evaluate.load("recall")
metric3 = evaluate.load("f1")
#metric4 = evaluate.load("accuracy")


In [24]:
example=dataset['train'][4]

In [25]:
labels=example['labels']

In [26]:
labels

[13, 5, 11, 6, 1, 9]

In [27]:
metric1.compute(predictions=labels, references=labels,average='micro')

{'precision': 1.0}

In [40]:
metric = load_metric("seqeval")

/tmp/ipykernel_23179/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/opt/mamba/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [41]:
import numpy as np

label_list=literal2tag
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

"recall": metric2.compute(predictions=Value(true_predictions), references=Value(true_labels),average='micro'),
        "f1": metric3.compute(predictions=Value(true_predictions), references=Value(true_labels),average='micro'),
        "accuracy": metric4.compute(predictions=Value(true_predictions), references=Value(true_labels)),

In [47]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer_french,
    compute_metrics=compute_metrics
)

In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 